In [ ]:
#| default_exp vanity

# Vanity addresses

> Use the notifyr bot to generate vanity addresses and get notified by DM when they are done.

## Finds vanity addresses
Here is [an interesting article by Kris Constable about vanity addresses](https://krisconstable.com/generating-a-key-pair-with-nostr/). This is a great usecase for our notifyr decorator because calculating vanity addresses can be quite slow if we are looking for long words.

This module will let you look for an `npub` or a `hex` vanity address that starts with a particular pattern.

In [ ]:
#| export

import time
import secrets
import secp256k1
from typing import Union
from nostrfastr.nostr import PrivateKey
from nostrfastr.notifyr import notifyr
from nostr import bech32

Make the functions to guess public keys - we are not using the classes from `python-nostr` to avoid carrying the overhead. The method of generating the keys is the exact same as the `python-nostr` key classes

In [ ]:
#| export

def guess_bytes():
    privkey_bytes = secrets.token_bytes(32) 
    sk = secp256k1.PrivateKey(privkey_bytes)
    pubkey_bytes = sk.pubkey.serialize()[1:]
    return privkey_bytes, pubkey_bytes

def guess_bech32(startswith=''):
    privkey_bytes, pubkey_bytes = guess_bytes()
    converted_bits = bech32.convertbits(pubkey_bytes, 8, 5)
    npub = bech32.bech32_encode("npub", converted_bits, bech32.Encoding.BECH32)
    if npub.startswith(startswith):
        return privkey_bytes.hex(), npub
    else:
        return None, None

def guess_hex(startswith=''):
    privkey_bytes, pubkey_bytes = guess_bytes()
    privkey_hex = privkey_bytes.hex()
    pubkey_hex = pubkey_bytes.hex()
    if pubkey_hex.startswith(startswith):
        return privkey_bytes.hex(), pubkey_bytes.hex()
    else:
        return None, None

In [ ]:
privkey_hex, npub = guess_bech32()
assert PrivateKey.from_hex(privkey_hex).public_key.bech32() == npub

In [ ]:
privkey_hex, pubkey_hex = guess_hex()
assert PrivateKey.from_hex(privkey_hex).public_key.hex() == pubkey_hex

We can make some functions to check the guess rate (hashrate)

In [ ]:
#| export

def time_guess(guesser):
    start = time.perf_counter()
    pub = guesser(startswith=' ')
    if pub is None: # this is just replicating what we will actually be doing
        pass
    end = time.perf_counter()
    interval = end - start
    return interval

def get_guess_rate(guesser, n_guesses=1e4):
    n_guesses = int(n_guesses)
    t = sum([time_guess(guesser) for _ in range(n_guesses)]) / n_guesses
    guesses_per_second = 1 / t
    return guesses_per_second

In [ ]:
get_guess_rate(guess_hex)

9855.1432254524

In [ ]:
get_guess_rate(guess_bech32)

3561.309377016123

And use our hashrate to make some estimates of how quickly we can find certain vanity addresses

In [ ]:
#| export

import math

In [ ]:
#| export

def expected_guesses_by_char(options: Union[str,list], num_char: int) -> float:
    """return an average number of guesses it would take to guess
    a pattern based on the number of characters in the pattern and
    the number of character options in the random output

    Parameters
    ----------
    options : list or str
        a set of characters as a str or list that are options for
        guessing
    num_char : int
        the number of characters in the pattern

    Returns
    -------
    float
        the expected number of guesses required to match the pattern
    """
    p = 1 / len(options)
    return (p ** -num_char - 1)/ (1 - p)

def expected_chars_by_time(options: Union[str,list], num_guesses: int) -> float:
    """the length of pattern you might expect to be able to guess given a
    certain number of guesses.

    Parameters
    ----------
    options : list or str
        a set of characters as a str or list that are options for
        guessing
    num_guesses : int
        the total number of guesses at a pattern

    Returns
    -------
    float
        th
    """
    p = 1 / len(options)
    n = - math.log(1 + (num_guesses * (1 - p))) / math.log(p)
    return n

def expected_time(options: Union[str,list], num_char: int, time_per_guess: float) -> float:
    """the expected amount of time it would take to guess a pattern with a certain
    length based on the average time per guess and the character options

    Parameters
    ----------
    options : list or str
        a set of characters as a str or list that are options for
        guessing
    num_char : int
        the number of characters in the pattern
    time_per_guess : float
        averge time per guess in seconds

    Returns
    -------
    float
        the expected amount of time needed to guess the pattern
    """
    n_guess = expected_guesses_by_char(options, num_char)
    time_seconds = n_guess * time_per_guess
    return time_seconds


In [ ]:
#| export

hex_chars = 'abcdef0123456789'
npub_chars = '023456789acdefghjklmnpqrstuvwxyz'

def average_char_by_time(options: Union[str,list], time_per_guess: float) -> None:
    """print an average number of characters you would expect to be
    able to guess for certain time periods based on character options
    and the average time per guess

    Parameters
    ----------
    options : list or str
        a set of characters as a str or list that are options for
        guessing
    time_per_guess : float
        the average time elapsed per guess
    """
    seconds_in_month = 60 * 60 * 24 * 30.5
    seconds_in_day = 60 * 60 * 24
    seconds_in_hour = 60 * 60
    seconds_in_minute = 60

    guesses_per_month = seconds_in_month / time_per_guess
    guesses_per_day = seconds_in_day / time_per_guess
    guesses_per_hour = seconds_in_hour / time_per_guess
    guesses_per_minute = seconds_in_minute / time_per_guess
    guesses_per_second = 1 / time_per_guess

    guesses = [guesses_per_second, guesses_per_minute,\
               guesses_per_hour, guesses_per_day, guesses_per_month]

    expected_chars = [expected_chars_by_time(options, g) for g in guesses]
    results = zip(['one second', 'one minute', 'one hour', 'one day', 'one month'],
                   expected_chars)
    for t, c in results:
        print(f'In {t} you can expect to get {c} characters on average')

def average_time_by_char(options: Union[str,list], time_per_guess: float) -> None:
    """print an average elapsed time for a range of pattern lengths

    Parameters
    ----------
    options : Union[str,list]
        a set of characters as a str or list that are options for
        guessing
    time_per_guess : float
        the average time elapsed per guess
    """
    for n in range(20):
        n += 1
        t = expected_time(options, n, time_per_guess)
        print(f'{n} characters: it might take {t} seconds')



In [ ]:
time_per_guess = 1 / get_guess_rate(guess_bech32)
average_char_by_time(npub_chars, time_per_guess)

In one second you can expect to get 2.3971816341588226 characters on average
In one minute you can expect to get 3.57848979484979 characters on average
In one hour you can expect to get 4.759866747853931 characters on average
In one day you can expect to get 5.676859229056948 characters on average
In one month you can expect to get 6.663006695772995 characters on average


In [ ]:
time_per_guess = 1 / get_guess_rate(guess_hex)
average_char_by_time(hex_chars, time_per_guess)

In one second you can expect to get 3.2573614997061533 characters on average
In one minute you can expect to get 4.734041727034478 characters on average
In one hour you can expect to get 6.210763668868103 characters on average
In one day you can expect to get 7.357004282563511 characters on average
In one month you can expect to get 8.589688616471262 characters on average


In [ ]:
average_time_by_char(hex_chars, time_per_guess)

1 characters: it might take 0.001794275524513796 seconds
2 characters: it might take 0.03050268391673453 seconds
3 characters: it might take 0.48983721819226633 seconds
4 characters: it might take 7.839189766600775 seconds
5 characters: it might take 125.4288305411369 seconds
6 characters: it might take 2006.863082933715 seconds
7 characters: it might take 32109.811121214967 seconds
8 characters: it might take 513756.97973371495 seconds
9 characters: it might take 8220111.677533715 seconds
10 characters: it might take 131521786.84233372 seconds
11 characters: it might take 2104348589.4791338 seconds
12 characters: it might take 33669577431.667934 seconds
13 characters: it might take 538713238906.6887 seconds
14 characters: it might take 8619411822507.021 seconds
15 characters: it might take 137910589160112.34 seconds
16 characters: it might take 2206569426561797.5 seconds
17 characters: it might take 3.530511082498876e+16 seconds
18 characters: it might take 5.6488177319982016e+17 seco

In [ ]:
#| export

def expected_performance():
    print(
        '''This is a random guessing process - estimations are an average, but the actual
        time it takes to find a key could be significantly more or less than the estimate!
        Please keep that in mind when choosing an option.
        ''')
    print('hex:')
    time_per_guess_hex = 1 / get_guess_rate(guesser=guess_hex)
    average_char_by_time(hex_chars, time_per_guess_hex)
    print('\n')
    average_time_by_char(hex_chars, time_per_guess_hex)
    print('\n')

    print('npub:')
    time_per_guess_bech32 = 1 / get_guess_rate(guesser=guess_bech32)
    average_char_by_time(npub_chars, time_per_guess_bech32)
    print('\n')
    average_time_by_char(npub_chars, time_per_guess_bech32)
    print('\n')

    

In [ ]:
expected_performance()

This is a random guessing process - estimations are an average, but the actual
        time it takes to find a key could be significantly more or less than the estimate!
        Please keep that in mind when choosing an option.
        
hex:
In one second you can expect to get 3.2328374922325778 characters on average
In one minute you can expect to get 4.709514734593529 characters on average
In one hour you can expect to get 6.1862366266715405 characters on average
In one day you can expect to get 7.332477239558768 characters on average
In one month you can expect to get 8.565161573432533 characters on average


1 characters: it might take 0.0019205368107184768 seconds
2 characters: it might take 0.03264912578221411 seconds
3 characters: it might take 0.5243065493261442 seconds
4 characters: it might take 8.390825326029026 seconds
5 characters: it might take 134.2551257532751 seconds
6 characters: it might take 2148.0839325892125 seconds
7 characters: it might take 34369.344841964215 s

Lets make the function...

In [ ]:
#| export

def gen_vanity_pubkey(startswith: str, style='hex') -> PrivateKey:
    """randomly generate private keys until one matches the desire
    startswith for an npub or hex

    Parameters
    ----------
    startswith : str
        characters that the public key should start with. More chars
        means longer run time
    style : str, optional
        'npub' or 'hex' - npub is more commonly displayed on apps
        while hex is the true base private key with no encoding,
        by default 'hex'

    Returns
    -------
    PrivateKey
        returns a private key object
    """
    pubkey = None
    if style == 'npub':
        if not all(c in npub_chars for c in startswith):
            raise ValueError(f'character of selection not '
                              'in npub pattern ({npub_chars})')
        time_per_guess = 1 / get_guess_rate(guess_bech32)
        t = expected_time(npub_chars, len(startswith), time_per_guess)
        startswith = f'npub1{startswith}'
    else:
        if not all(c in hex_chars for c in startswith):
            raise ValueError(f'character of selection not in '
                              'hex pattern ({hex_chars})')
        time_per_guess = 1 / get_guess_rate(guess_hex)
        t = expected_time(hex_chars, len(startswith), time_per_guess)
    print(f'It might take {t} seconds to find a {style} pubkey that starts with '
          f'{startswith}. Note that this is a very rough estimate and due '
          'to the random nature of finding vanity keys it could take MUCH '
          'longer.')
    while pubkey is None:
        if style == 'npub':
            privkey_hex, pubkey = guess_bech32(startswith=startswith)
        else:
            privkey_hex, pubkey = guess_hex(startswith=startswith)
    return PrivateKey.from_hex(privkey_hex)

In [ ]:
from fastcore.test import test_fail

Make sure we don't allow characters that will never happen - test that these cases fail

In [ ]:
fail = lambda _: gen_vanity_pubkey(startswith='b', style='npub')
test_fail(fail)
fail = lambda _: gen_vanity_pubkey(startswith='g', style='hex')
test_fail(fail)


Generate a couple npubs!

In [ ]:
vanity_private_key_npub = gen_vanity_pubkey(startswith='23', style='npub')
vanity_private_key_hex = gen_vanity_pubkey(startswith='23', style='hex')

It might take 0.28460781893972303 seconds to find a npub pubkey that starts with npub123. Note that this is a very rough estimate and due to the random nature of finding vanity keys it could take MUCH longer.
It might take 0.025928018985502423 seconds to find a hex pubkey that starts with 23. Note that this is a very rough estimate and due to the random nature of finding vanity keys it could take MUCH longer.


And make sure it worked...

In [ ]:
assert vanity_private_key_npub.public_key.bech32().startswith('npub123')
assert vanity_private_key_hex.public_key.hex().startswith('23')

Now we can also make a version of this that notifies you.

In [ ]:
#| export

vanity_notifyr = notifyr(gen_vanity_pubkey)

Remember that if you want a vanity notifyr that will go to a different address than the one you find in `vanity_notifyr.notifyr_privkey_hex` you can create your own notifyr like so:
```python
new_vanity_notifyr = notifyr(gen_vanity_pubkey, recipient_address=your_address)
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()